### 1.1 Prior Installations and Configurations

pip install openai
pip install langchain
pip install langchain-openai
pip install langchain_community

Register for a API key @ https://platform.openai.com/
Your Profile -> User API Keys
Copy the key to a text file. 
Make sure it's kept secure
Have some $5 credits as well

### 1.2 Basic Invocation

In [2]:
from langchain_openai import OpenAI

f = open(r"C:\Users\mindf\Desktop\current-work\openai-api-key-purushotham.txt")
apikey = f.read()
f.close()

llm = OpenAI(api_key=apikey)

#### A simple way to get text auto complete

In [ ]:
print(llm.invoke('Here is a fun fact about Pluto:'))

#### Use generate for full output:

In [ ]:
# NEEDS TO BE A LIST, EVEN FOR JUST ONE STRING
result = llm.generate(['Here is a fun fact about Pluto:',
                     'Here is a fun fact about Mars:']
                     )

In [ ]:
result.schema()

In [ ]:
result.llm_output

### 1.3 Chat Models

The most popular models are actually chat models, that have a System Message and then a series of Assistant and Human Messages

In [ ]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(openai_api_key=apikey)

In [ ]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
result = chat.invoke([HumanMessage(content="Can you tell me a fact about Earth?")])
result.content

In [ ]:
result = chat.invoke([SystemMessage(content='You are a very rude teenager who only wants to party and not answer questions'),
               HumanMessage(content='Can you tell me a fact about Earth?')])
result.content

In [ ]:
# NEEDS TO BE A LIST!
result = chat.generate(
                [
                    [  SystemMessage(content='You are a University Professor'),
                       HumanMessage(content='Can you tell me a fact about Earth?') ]
                ]
)
result.llm_output

In [ ]:
result.generations[0][0].text

### 1.4 Extra Paramters and Arguments

In [ ]:
result = chat.invoke([HumanMessage(content='Can you tell me a fact about Earth?')],
                 temperature=2,presence_penalty=1,max_tokens=100)
result.content

### 2.0 Understanding Prompt Templates

#### 2.1 Input Variables

In [ ]:
from langchain import PromptTemplate

# An example prompt with no input variables
no_input_prompt = PromptTemplate(input_variables=[], template="Tell me a fact")
no_input_prompt.format()

In [ ]:
# An example prompt with one input variable
one_input_prompt = PromptTemplate(input_variables=["topic"], template="Tell me a fact about {topic}.")
# Notice how the stirng "topic" gets automatically converted to a parameter name, very convienent! 
one_input_prompt.format(topic="Mars")
# -> "Tell me a fact about Mars"

In [ ]:
# An example prompt with multiple input variables
multiple_input_prompt = PromptTemplate(
    input_variables=["topic", "level"], 
    template="Tell me a fact about {topic} for a student {level} level."
)
multiple_input_prompt.format(topic='Mars',level='8th Grade')

#### 2.2 Prompt Templates

Chat models require a list of chat messages called a prompt, which is different from a raw string that you would input into a language model. Each message in the prompt is associated with a role, such as AI, human, or system.

For instance, when using the OpenAI Chat Completion API, a chat message can be assigned the role of AI, human, or system. The model is designed to pay closer attention to instructions provided in system chat messages.

To simplify the process of constructing and working with prompts, LangChain offers various prompt templates. It is highly recommended to utilize these chat-related prompt templates instead of PromptTemplate when interacting with chat models. This will allow you to fully harness the potential of the underlying chat model and enhance your experience.

We will favor these models in the course due to upcoming changes in the OpenAI ecosystem where chat agents will be favored over text completion models.

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
system_template="You are an AI recipe assistant that specializes in {dietary_preference} dishes that can be prepared in {cooking_time}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [ ]:
system_message_prompt.input_variables

In [ ]:
human_template="{recipe_request}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
human_message_prompt.input_variables

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [ ]:
chat_prompt.input_variables

In [ ]:
# get a chat completion from the formatted messages
chat_prompt.format_prompt(cooking_time="15 min", dietary_preference="Vegan", recipe_request="Quick Snack").to_messages()

In [ ]:
request = chat_prompt.format_prompt(cooking_time="15 min", dietary_preference="Vegan", recipe_request="Quick Snack").to_messages()

In [ ]:
result = chat.invoke(request)

In [ ]:
print(result.content)

#### 2.3 Few Shot Prompting

In [ ]:
template = "You are a helpful assistant that translates complex legal terms into plain and understandable terms."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

In [ ]:
legal_text = "The provisions herein shall be severable, and if any provision or portion thereof is deemed invalid, illegal, or unenforceable by a court of competent jurisdiction, the remaining provisions or portions thereof shall remain in full force and effect to the maximum extent permitted by law."
example_input_one = HumanMessagePromptTemplate.from_template(legal_text)

# Use this for creating example AI prompt
plain_text = "The rules in this agreement can be separated. If a court decides that one rule or part of it is not valid, illegal, or cannot be enforced, the other rules will still apply and be enforced as much as they can under the law."
example_output_one = AIMessagePromptTemplate.from_template(plain_text)

In [ ]:
human_template = "{legal_text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, example_input_one, example_output_one, human_message_prompt]
)

In [ ]:
example_text = "The grantor, being the fee simple owner of the real property herein described, conveys and warrants to the grantee, his heirs and assigns, all of the grantor's right, title, and interest in and to the said property, subject to all existing encumbrances, liens, and easements, as recorded in the official records of the county, and any applicable covenants, conditions, and restrictions affecting the property, in consideration of the sum of [purchase price] paid by the grantee."
request = chat_prompt.format_prompt(legal_text=example_text).to_messages()

In [ ]:
result = chat.invoke(request)

In [ ]:
print(result.content)

### 3.0 Exercise 

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from datetime import datetime
from langchain_openai import OpenAI
from langchain.output_parsers import DatetimeOutputParser
from langchain.chat_models import ChatOpenAI

In [ ]:
class HistoryQuiz():
    
    def create_history_question(self,topic):
        '''
        This method should output a historical question about the topic that has a date as the correct answer.
        For example:
        
            "On what date did World War 2 end?"
            
        '''
       
        return question
    
    def get_AI_answer(self,question):
        '''
        This method should get the answer to the historical question from the method above.
        Note: This answer must be in datetime format! Use DateTimeOutputParser to confirm!
        
        September 2, 1945 --> datetime.datetime(1945, 9, 2, 0, 0)
        '''
         
        
        return correct_datetime
    
    def get_user_answer(self,question):
        '''
        This method should grab a user answer and convert it to datetime. It should collect a Year, Month, and Day.
        You can just use input() for this.
        '''
        

        
        return user_datetime
        
        
    def check_user_answer(self,user_answer,ai_answer):
        '''
        Should check the user answer against the AI answer and return the difference between them
        '''
        # print or return the difference between the answers here!
        pass
        

In [ ]:
quiz_bot = HistoryQuiz()
question = quiz_bot.create_history_question(topic='World War 2')
question

In [ ]:
ai_answer = quiz_bot.get_AI_answer(question)
ai_answer

In [ ]:
user_answer = quiz_bot.get_user_answer(question)
user_answer

In [ ]:
quiz_bot.check_user_answer(user_answer,ai_answer)

### 4.0 Document Loaders

There are many other types of Documents that can be loaded in. You can see all the document loaders available here: https://python.langchain.com/docs/modules/data_connection/document_loaders/

Keep in mind many Loaders are dependent on other libraries, meaning issues in those libraries can end up breaking the Langchain loaders.

#### 4.1 CSV Loader

In [ ]:
from langchain.document_loaders import CSVLoader

In [ ]:
loader = CSVLoader(r'C:\Users\mindf\Desktop\sapient-us\langchain\penguins.csv')
data = loader.load()

In [ ]:
print(data[0].page_content)

#### 4.2 HTML 

In [ ]:
from langchain.document_loaders import BSHTMLLoader

In [ ]:
loader = BSHTMLLoader(r'C:\Users\mindf\Desktop\sapient-us\langchain\some_website.html')
data = loader.load()
data

#### 4.3 PDF

In [ ]:
# !pip install pypdf

In [ ]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
loader = PyPDFLoader(r'C:\Users\mindf\Desktop\sapient-us\langchain\report.pdf')
pages = loader.load_and_split()

In [ ]:
print(pages[0].page_content)

#### 4.4 Document Tranformations: Split by Character, split by tokens

In [ ]:
with open(r'C:\Users\mindf\Desktop\sapient-us\langchain\FDR_State_of_Union_1944.txt') as file:
    speech_text = file.read()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
text_splitter = CharacterTextSplitter(separator="\n\n",chunk_size=1000) #1000 is default value

In [ ]:
texts = text_splitter.create_documents([speech_text])
print(type(texts))
print('\n')
print(texts[0])

In [ ]:
type(texts[0])

In [ ]:
#!pip install tiktoken

In [ ]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size = 500) #now chunk size is a hard length based on tokens

In [ ]:
texts = text_splitter.split_text(speech_text)

In [ ]:
texts[0]

#### 4.5 Text Embeddings 

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

In [ ]:
embeddings = OpenAIEmbeddings(api_key=apikey)

In [ ]:
text = "Some normal text to send to OpenAI to be embedded into a N dimensional vector"

In [ ]:
embedded_text = embeddings.embed_query(text)

In [ ]:
embedded_text[0]

### 5.0 Vector Store

##### We can save the embeddings into a Vector store - Chroma

In [ ]:
#!pip install langchain_chroma

In [ ]:
import chromadb
print(chromadb.__version__)

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

##### Load the document and split(still recommended even if under the context window)

In [ ]:
# load the document and split it into chunks
loader = TextLoader(r'C:\Users\mindf\Desktop\sapient-us\langchain\FDR_State_of_Union_1944.txt')
documents = loader.load()

In [ ]:
# split it into chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

##### Connect to OpenAI for Embeddings

In [ ]:
embedding_function = OpenAIEmbeddings(api_key=apikey)

##### Pass Embeddings and Docs into Chroma

In [ ]:
# load it into Chroma
db = Chroma.from_documents(docs, embedding_function,persist_directory=r'C:\Users\mindf\Desktop\sapient-us\langchain-course\speech_embedding_db')

##### Save the new embeddings to the disk

In [ ]:
# Helpful to force a save
db.persist()

##### Loading embeddings from the disk

In [ ]:
persist_directory=r'C:\Users\mindf\Desktop\sapient-us\langchain-course\speech_embedding_db'
db_connection = Chroma(persist_directory=persist_directory, embedding_function=embedding_function)

In [ ]:
new_doc = "What did FDR say about the cost of food law?"
docs = db_connection.similarity_search(new_doc)

In [ ]:
print(docs[0].page_content)

##### Adding a new document

In [ ]:
# load the document and split it into chunks
loader = TextLoader(r"C:\Users\mindf\Desktop\sapient-us\langchain-course\Lincoln_State_of_Union_1862.txt")
documents = loader.load()

In [ ]:
# split it into chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

In [ ]:
# load it into Chroma
db = Chroma.from_documents(docs, embedding_function,persist_directory=persist_directory)

In [ ]:
docs = db.similarity_search('slavery')

In [ ]:
docs[0].page_content

### 5.1 Vector Store Retriever

In [ ]:
from langchain.document_loaders import WikipediaLoader
from langchain_chroma import Chroma

In [ ]:
embedding_function = OpenAIEmbeddings(api_key=apikey)

In [ ]:
db_connection = Chroma(persist_directory=r'C:\Users\mindf\Desktop\sapient-us\langchain-course\mk_ultra',embedding_function=embedding_function)

In [ ]:
retriever = db_connection.as_retriever()

In [ ]:
search_kwargs = {"score_threshold":0.8,"k":4}
docs = retriever.invoke("President",search_kwargs=search_kwargs)

In [ ]:
docs[0].page_content

### 6.0 Exercise : Vector Stores

###  Data Connections Exercise

#### Ask a Legal Research Assistant Bot about the US Constitution

Let's revisit our first exercise and add offline capability using ChromaDB. Your function should do the following:

* Read the US_Constitution.txt file inside the some_data folder
* Split this into chunks (you choose the size)
* Write this to a ChromaDB Vector Store
* Use Context Compression to return the relevant portion of the document to the question

In [ ]:
# Build a sample vectorDB
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor 

In [ ]:
def us_constitution_helper(question):
    '''
    Takes in a question about the US Constitution and returns the most relevant
    part of the constitution. Notice it may not directly answer the actual question!
    
    Follow the steps below to fill out this function:
    '''

    persist_directory=r"C:\Users\mindf\Desktop\sapient-us\langchain-course"
    
    # PART ONE:
    # LOAD "/US_Constitution in a Document object
    loader = TextLoader(r"C:\Users\mindf\Desktop\sapient-us\langchain-course\US_Constitution.txt")
    documents = loader.load()
    
    # PART TWO
    # Split the document into chunks (you choose how and what size)
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
    docs = text_splitter.split_documents(documents)
    
    # PART THREE
    # EMBED THE Documents (now in chunks) to a persisted ChromaDB
    embedding_function = OpenAIEmbeddings(api_key=apikey)
    db = Chroma.from_documents(docs, embedding_function,persist_directory=persist_directory)
    db.persist()

    # PART FOUR
    # Use ChatOpenAI and ContextualCompressionRetriever to return the most
    # relevant part of the documents.

    # results = db.similarity_search("What is the 13th Amendment?")
    # print(results[0].page_content) # NEED TO COMPRESS THESE RESULTS!
    llm = ChatOpenAI(temperature=0, api_key=apikey)
    compressor = LLMChainExtractor.from_llm(llm)

    compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, 
                                                           base_retriever=db.as_retriever())

    compressed_docs = compression_retriever.invoke(question)

    return compressed_docs[0].page_content

In [ ]:
print(us_constitution_helper("What is the 13th Amendment?"))

### 7.0 Chains

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains.sequential import SequentialChain

In [ ]:
llm = ChatOpenAI(api_key=apikey)

##### Simple Chain

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

In [ ]:
human_message_prompt = HumanMessagePromptTemplate.from_template(
        "Make up a funny company name for a company that produces {product}"
    )
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])


In [ ]:
chain = chat_prompt_template | llm

In [ ]:
print(chain.invoke(input="Computers").content)

##### Sequntial Chain

In [ ]:
template1 = "Give a summary of this employee's performance review:\n{review}"
prompt1 = ChatPromptTemplate.from_template(template1)
chain_1 = prompt1|llm

In [ ]:
template2 = "Identify key employee weaknesses in this review summary:\n{review_summary}"
prompt2 = ChatPromptTemplate.from_template(template2)
chain_2 = prompt2|llm

In [ ]:
template3 = "Create a personalized plan to help address and fix these weaknesses:\n{weaknesses}"
prompt3 = ChatPromptTemplate.from_template(template3)
chain_3 = prompt3|llm

In [ ]:
seq_chain = chain_1|chain_2|chain_3

In [ ]:
employee_review = '''
Employee Information:
Name: Joe Schmo
Position: Software Engineer
Date of Review: July 14, 2023

Strengths:
Joe is a highly skilled software engineer with a deep understanding of programming languages, algorithms, and software development best practices. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.

One of Joe's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.

Joe consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.

Another notable strength is Joe's adaptability. He has shown great flexibility in handling changing project requirements and learning new technologies. This adaptability allows him to seamlessly transition between different projects and tasks, making him a valuable asset to the team.

Joe's problem-solving skills are exceptional. He approaches issues with a logical mindset and consistently finds effective solutions, often thinking outside the box. His ability to break down complex problems into manageable parts is key to his success in resolving issues efficiently.

Weaknesses:
While Joe possesses numerous strengths, there are a few areas where he could benefit from improvement. One such area is time management. Occasionally, Joe struggles with effectively managing his time, resulting in missed deadlines or the need for additional support to complete tasks on time. Developing better prioritization and time management techniques would greatly enhance his efficiency.

Another area for improvement is Joe's written communication skills. While he communicates well verbally, there have been instances where his written documentation lacked clarity, leading to confusion among team members. Focusing on enhancing his written communication abilities will help him effectively convey ideas and instructions.

Additionally, Joe tends to take on too many responsibilities and hesitates to delegate tasks to others. This can result in an excessive workload and potential burnout. Encouraging him to delegate tasks appropriately will not only alleviate his own workload but also foster a more balanced and productive team environment.
'''

In [ ]:
results = seq_chain.invoke(employee_review)

In [ ]:
print(results.content)

### 8.0 Exercise - Chains

####  Chains Exercise - Solution

#### TASK:
Fill out the function below that takes in a string input Customer Support email that could be written in any language. The function will then detect the language, translate the email, and provide a summary.

Fill out the function below using a Sequential Chain, the function should do the following:

1. Detect the language the email is written in
2. Translate the email from detected language to English
3. Return a summary of the translated email

Note: The Function should return a dictionary that contains all three of these outputs!

In [ ]:
spanish_email = open(r'C:\Users\mindf\Desktop\sapient-us\langchain-course\spanish_customer_email.txt', encoding="latin-1").read()

In [ ]:
print(spanish_email)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [ ]:
def translate_and_summarize(email):
    """
    Translates an email written in a detected language to English and generates a summary.

    Args:
        email (str): The email to be processed and translated.

    Returns:
        dict: A dictionary containing the following keys:
            - 'language': The language the email was written in.
            - 'translated_email': The translated version of the email in English.
            - 'summary': A short summary of the translated email.

    Raises:
        Exception: If any error occurs during the LLM chain execution.

    Example:
        email = "Hola, ¿cómo estás? Espero que todo vaya bien."
        result = translate_and_summarize(email)
        print(result)
        # Output:
        # {
        #     'language': 'Spanish',
        #     'translated_email': 'Hello, how are you? I hope everything is going well.',
        #     'summary': 'A friendly greeting and a wish for well-being.'
        # }
    """
    # Create Model
    llm = ChatOpenAI(api_key=apikey)
    
    # CREATE A CHAIN THAT DOES THE FOLLOWING:
    
    # Detect Language
    template1 = "Return the language this email is written in:\n{email}.\nONLY return the language it was written in."
    prompt1 = ChatPromptTemplate.from_template(template1)
    chain_1 = prompt1|llm
    
    # Translate from detected language to English
    template2 = "Translate this email from {language} to English. Here is the email:\n"+email
    prompt2 = ChatPromptTemplate.from_template(template2)
    chain_2 = prompt2|llm
    
    # Return English Summary AND the Translated Email
    template3 = "Create a short summary of this email:\n{translated_email}"
    prompt3 = ChatPromptTemplate.from_template(template3)
    chain_3 = prompt3|llm

    language_chain = chain_1
    translation_chain = language_chain|chain_2
    seq_chain = translation_chain|chain_3
    
    return language_chain.invoke(email), translation_chain.invoke(email), seq_chain.invoke(email)

In [ ]:
result = translate_and_summarize(spanish_email)

In [ ]:
result[0]

In [ ]:
result[1]

In [ ]:
result[2]

### 9.0 Memory

##### Chat Message History

In [ ]:
from langchain.memory import ChatMessageHistory
history = ChatMessageHistory()
history.add_user_message("Hello, nice to meet you.")
history.add_ai_message("Nice to meet you too!")

In [ ]:
history.messages

##### Conversation Buffer Memory

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:
llm = ChatOpenAI(temperature=0.0, api_key=apikey)
memory = ConversationBufferMemory()

In [ ]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [ ]:
conversation.invoke(input="Hello, nice to meet you!")

In [ ]:
conversation.invoke(input="Tell me about the Einstein-Szilard Letter ")

##### Saving and Loading Memory

Best Source We've Found: https://stackoverflow.com/questions/75965605/how-to-persist-langchain-conversation-memory-save-and-load

In [ ]:
conversation.memory

In [ ]:
import pickle
pickled_str = pickle.dumps(conversation.memory)

In [ ]:
with open('memory.pkl','wb') as f:
    f.write(pickled_str)

In [ ]:
new_memory_load = open('memory.pkl','rb').read()

In [ ]:
llm = ChatOpenAI(temperature=0.0, api_key=apikey)
reload_conversation = ConversationChain(
    llm=llm, 
    memory = pickle.loads(new_memory_load),
    verbose=True
)

In [ ]:
reload_conversation.memory.buffer

### 10.0 Agents

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

This is the chain responsible for deciding what step to take next. This is usually powered by a language model, a prompt, and an output parser.

There are several key concepts to understand when building agents: Agents, AgentExecutor, Tools, Toolkits. For an in depth explanation, please check out this conceptual guide: https://python.langchain.com/v0.1/docs/modules/agents/concepts/

##### Load LLM

In [169]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key=apikey)

##### Define Tools

In [ ]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

get_word_length.invoke("abc")

In [171]:
tools = [get_word_length]

##### Create a prompt

In [172]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

##### Bind tools to LLM

In [173]:
llm_with_tools = llm.bind_tools(tools)

##### Create the agent

Putting those pieces together, we can now create the agent. We will import two last utility functions: a component for formatting intermediate steps (agent action, tool output pairs) to input messages that can be sent to the model, and a component for converting the output message into an agent action/agent finish.

In [187]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [188]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
list(agent_executor.stream({"input": "How many letters in the word purushotham"}))

##### If we compare this to base LLM, we can see that LLM alone struggles

In [ ]:
llm.invoke("How many letters in the word purushotham")

### 11. Simple Project

Imagine you are just starting out with an ice-cream business and want to learn everything about ice-creams (your ideal customers, unique flavor combinations, easy ice-cream recipes etc.). Let’s call our ice-cream assistant Scoopsie. We’ll develop our chatbot using LangChain and OpenAI’s text completion model. 

In [3]:
from langchain_openai import OpenAI

llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0.7, api_key=apikey)

Next, we need to create a LLM chain. In LangChain, LLM chains represent a higher-level abstraction for interacting with language models. While we can use the direct LLM interface in our simple chatbot, the LLMChain interface wraps an LLM to add additional functionality. With chains, we can have prompt formatting and input/output parsing, and they are used extensively by higher level LangChain tools. For our simple chatbot, we will use the LLMChain, and pass it the model object we created, alongwith our ice-cream assistant prompt template.

In [6]:
from langchain.prompts import PromptTemplate

ice_cream_assistant_template = """
You are an ice cream assistant chatbot named "Scoopsie". Your expertise is 
exclusively in providing information and advice about anything related to ice creams. This includes flavor combinations, ice cream recipes, and general 
ice cream-related queries. You do not provide information outside of this 
scope. If a question is not about ice cream, respond with, "I specialize only in ice cream related queries." 
Question: {question} 
Answer:"""

ice_cream_assistant_prompt_template = PromptTemplate(
    input_variables=["question"],
    template=ice_cream_assistant_template
)

In [ ]:
from langchain.chains import LLMChain

llm_chain = LLMChain(llm=llm, prompt=ice_cream_assistant_prompt_template)

Our simple chatbot’s code is mostly complete. We just need to create an entry point for our script aka the main function. We also need to be able to query our model. This is done using the invoke function with the llm_chain object. Putting together everything we have done so far in our chatbot.py script, you will have something that looks like this:

In [ ]:
from langchain_openai import OpenAI
from langchain.chains import LLMChain


from dotenv import load_dotenv

load_dotenv()


llm_chain = LLMChain(llm=llm, prompt=ice_cream_assistant_prompt_template)


def query_llm(question):
    print(llm_chain.invoke({'question': question})['text'])


if __name__ == '__main__':
    query_llm("Who are you?")

In [ ]:
question = "I need chocolate ice-cream recipe"
print(llm_chain.invoke({'question': question})['text'])